# Pair Classifier Inference

In [ ]:
from transformers import (
    EsmTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)

import torch
import pandas as pd
import numpy as np

import datasets
from datasets import ClassLabel

from tqdm.notebook import tqdm
from pathlib import Path

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

## setup model & data paths

In [ ]:
# models and corresponding test data
pair_dict = {
    n: {
        # replace with actual paths to models
        "models": {
            "8M-Q":   f"5fold8M_QUARTER_itr{n}_Paired_class_Final",
            "8M-H":   f"5fold8M_HALF_itr{n}_Paired_class_Final",
            "8M-F":   f"5fold8M_FULL_itr{n}_Paired_class_Final",
            "35M-Q":  f"5fold35M_QUARTER_itr{n}_Paired_class_Final",
            "35M-H":  f"5fold35M_HALF_itr{n}_Paired_class_Final",
            "35M-F":  f"5fold35M_FULL_itr{n}_Paired_class_Final",
            "150M-Q": f"5fold150M_QUARTER_itr{n}_Paired_class_Final",
            "150M-H": f"5fold150M_HALF_itr{n}_Paired_class_Final",
            "150M-F": f"5fold150M_FULL_itr{n}_Paired_class_Final",
            "350M-Q": f"5fold350M_QUARTER_itr{n}_Paired_class_Final",
            "350M-H": f"5fold350M_HALF_itr{n}_Paired_class_Final",
            "350M-F": f"5fold350M_FULL_itr{n}_Paired_class_Final",
            "650M-Q": f"5foldNEW_650M_QUARTER_itr{n}_Paired_class_Final",
            "650M-H": f"5fold650M_HALF_itr{n}_Paired_class_Final",
            "650M-F": f"5fold650M_FULL_itr{n}_Paired_class_Final",
        },
        "data": f"../../data/SA_donors_stuff/train-test_splits/native-0_shuffled-1_test{n}.csv",
    }
for n in range(5)}


## check that paths exist

In [ ]:
def find_folder_path(start_string, search_dir='.'):
    base = Path(search_dir)
    for folder in base.iterdir():
        if folder.is_dir() and folder.name.startswith(start_string):
            return folder.resolve()
    raise FileNotFoundError(f"No folder starting with '{start_string}' found in '{base.resolve()}'.")

In [ ]:
for i in pair_dict.keys():
    for model_id, model_path in pair_dict[i]["models"].items():
        folder_path = find_folder_path(model_path, "../paired_clssification_models/")
        print(folder_path)

## run inference

In [ ]:
for i in tqdm(pair_dict.keys()):

    # load test data
    test_data = pd.read_csv(pair_dict[i]["data"])
    test_preds = test_data.copy() # for storing prediction metrics
    
    class_labels = ClassLabel(names=["native-pair", "shuffled-pair"])
    n_classes = len(class_labels.names)
    label2id = {"native-pair": 0, "shuffled-pair": 1}
    id2label = {0: "native-pair", 1: "shuffled-pair"}
    
    # make huggingface dataset
    dataset = datasets.Dataset.from_pandas(test_data)
    dataset = dataset.cast_column("label", class_labels)
    
    # filter for length (model has max length of 320 from training)
    def filter_long_sequences(item):
        return (len(item['h_sequence'])+len(item['l_sequence'])) <= 315 # allows 4 tokens (start, sep (which is 2 tokens long), end)
    filtered = dataset.filter(filter_long_sequences)
    
    # tokenizer
    tokenizer = EsmTokenizer.from_pretrained("facebook/esm2_t30_150M_UR50D")
    
    def preprocess_dataset(
        batch, 
        tokenizer=None, 
        separator="<cls><cls>",
        max_len=320
    ) -> list:
        """
        docstring
        """
        # tokenize the H/L sequence pair
        sequences = [h + separator + l for h, l in zip(batch["h_sequence"], batch["l_sequence"])]
        tokenized = tokenizer(sequences, padding="max_length", max_length=max_len)
        batch["input_ids"] = tokenized.input_ids
        batch["attention_mask"] = tokenized.attention_mask
        
        return batch
    
    # tokenize
    tokenized_dataset = filtered.map(
        preprocess_dataset,
        fn_kwargs={
            "tokenizer": tokenizer,
            "max_len": 320,
        },
        batched=True,
        remove_columns=["name", "h_sequence", "l_sequence", "donor"]
    )

    # load each model
    for model_id, model_path_str in pair_dict[i]["models"].items():
        print(f"{i}: {model_id}")
        model_path = find_folder_path(model_path_str, "../paired_clssification_models/")
        model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)
    
        # predict on test set and get metrics
        trainer = Trainer(
            model=model,
            tokenizer=tokenizer,
            args=TrainingArguments(output_dir="./", 
                                   report_to="none"), # to turn off wandb logging
            eval_dataset=tokenized_dataset,
        )
        logits, labels, metrics = trainer.predict(tokenized_dataset)
        probabilities = torch.softmax(torch.from_numpy(logits), dim=1).detach().numpy()[:, -1]
        predictions = np.argmax(logits, axis=1)
        
        del model # free up memory
        
        # categorize predictions
        pred_data = []
        for pred, prob, label, logit in zip(predictions, probabilities, labels, logits):
            if pred == label == 1:
                category = "true_positive"
            elif pred == label == 0:
                category = "true_negative"
            elif pred == 1 and label == 0:
                category = "false_positive"
            else:
                category = "false_negative"
            pred_data.append(
                {
                    # "label": label,
                    f"{model_id}_prediction": pred,
                    f"{model_id}_probability": prob,
                    f"{model_id}_category": category,
                    f"{model_id}_logits": logit,
                }
            )
        pred_df = pd.DataFrame(pred_data)
        
        # store predictive performance with its corresponding sequence
        test_preds = pd.concat([test_preds, pred_df], axis=1)
    
    # save as csv
    save_path = f"./results/all_predictions_itr{i}.csv"
    print(f"saving to csv: {save_path} \n-------------------\n\n")
    test_preds.to_csv(save_path, index=False)